In [1]:
import requests
import os

from bs4 import BeautifulSoup
from urllib.parse import urljoin

In [10]:
headers = {"User-Agent" : ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"),}

In [3]:
url = 'https://m.comic.naver.com/webtoon/list?titleId=769209&week=wed&sortOrder=ASC'

In [4]:
res = requests.get(url, headers=headers)
res

<Response [200]>

In [5]:
html = res.text
len(html)

41032

# 웹툰 제목, 링크 가져오기

In [6]:
soup = BeautifulSoup(html, 'html.parser')
soup


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://ssl.pstatic.net/static/m/comic/im/favicon/1804/webtoon_favicon_32x32.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://ssl.pstatic.net/static/m/comic/im/favicon/1804/ios_152x152.png" rel="apple-touch-icon-precomposed">
<title>윈드브레이커 :: 네이버 웹툰</title>
<meta content="윈드브레이커" property="og:title">
<meta content="https://shared-comic.pstatic.net/thumb/webtoon/602910/thumbnail/thumbnail_IMAG19_f07f7ef0-00d4-40b9-8b5b-187f507356dc.jpg" property="og:image">
<m

In [7]:
tag_list = soup.select(".section_episode_list a")
tag_list

[<a class="link" href="/webtoon/detail?titleId=602910&amp;no=1&amp;week=mon&amp;listSortOrder=ASC&amp;listPage=1" onclick="nclk_v2(event,'lst.episode','602910','1')">
 <div class="thumbnail">
 <img alt="00화" height="45" src="https://image-comic.pstatic.net/webtoon/602910/1/inst_thumbnail_20131206195920.jpg" width="76"/>
 </div>
 <div class="info">
 <strong class="title">
 <span class="name"><strong>00화</strong></span>
 </strong>
 <div class="detail">
 <span class="ico_comic ico_score"></span>
 <span class="score">9.92</span>
 <span class="date">13.12.08</span>
 </div>
 </div>
 </a>,
 <a class="link" href="/webtoon/detail?titleId=602910&amp;no=2&amp;week=mon&amp;listSortOrder=ASC&amp;listPage=1" onclick="nclk_v2(event,'lst.episode','602910','2')">
 <div class="thumbnail">
 <img alt="01화" height="45" src="https://image-comic.pstatic.net/webtoon/602910/2/inst_thumbnail_20131213221443.jpg" width="76"/>
 </div>
 <div class="info">
 <strong class="title">
 <span class="name"><strong>01화</str

In [9]:
wt_name = soup.select(".title")
wt_name = wt_name[0].text
wt_name

'윈드브레이커'

In [11]:
for a_tag in tag_list:
    # 에피소드
    ep_url = a_tag['href']
    if ep_url != '#':
        name_tag = a_tag.select('.name')[0]
        name = name_tag.text # 웹툰 제목
        
        ep_url = "https://comic.naver.com" + ep_url
        ep_url = urljoin(url, ep_url)
        
        # 각 에피소드 들어가기 위해 정의
        ep_res = requests.get(ep_url, headers = headers)
        ep_html = ep_res.text
        ep_soup = BeautifulSoup(ep_html, 'html.parser')
        ep_tag_list = ep_soup.select('.wt_viewer img')
        
        for idx, tag in enumerate(ep_tag_list, 1):
            # 파일 경로 정의
            ep_dir = wt_name + '/' + name
            filename = f'{wt_name}/{name}/{idx:03d}.jpg'
            image_url = tag['src']
            image_res = requests.get(tag['src'], headers = headers)
            assert image_res.status_code == 200, "서버 응답이 200이 아니에요!"
            
            # 폴더 생성
            os.makedirs(ep_dir, exist_ok=True)
            
            # 이미지 저장
            with open(filename, 'wb') as f:
                f.write(image_res.content)
        

FileNotFoundError: [Errno 2] No such file or directory: '윈드브레이커/28화 /001.jpg'

### 